<a href="https://colab.research.google.com/github/DineshY1011/US_Accident/blob/main/Milestone_3/Week_6/Day_27/Light_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### LightGBM Classifier Model Training

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 60.0 MB/s eta 0:00:00


In [5]:
# Load necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

In [6]:
# Load data (update the path as required)
df = pd.read_csv("/content/drive/MyDrive/US_Dataset/Cleaned_1M_Dataset/cleaned_us_accidents.csv")
# df = df.sample(100000, random_state=42)

# Separate features and target
target = 'Severity'
X = df.drop(columns=target)
y = df[target]

In [7]:
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()

In [8]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [10]:
clf_lgbm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', lgb.LGBMClassifier(
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        max_depth=-1,
        random_state=42,
        class_weight='balanced',
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

clf_lgbm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.392242 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16812
[LightGBM] [Info] Number of data points in the train set: 399393, number of used features: 7591
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Start_Lat', 'Start_Lng',
                                                   'Distance(mi)',
                                                   'Temperature(F)',
                                                   'Wind_Chill(F)',
                                                   'Humidity(%)',
                                                   'Pressure(in)',
                                                   'Visibility(mi)',
                                                   'Wind_Speed(mph)',
                                                   'Precipitation(in)',
                                                   'Amenity', 'Bump',
                                                   'Crossing', 'Gi...
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Start_Time', 'End_Time',
                                                   'City', 'County', 'State',
                                                   'Zipcode', 'Wind_Direction',
                                                   'Weather_Condition',
                                                   'Sunrise_Sunset',
                                                   'Civil_Twilight'])])),
                ('classifier',
                 LGBMClassifier(class_weight='balanced', colsample_bytree=0.8,
                                learning_rate=0.05, n_estimators=300,
                                random_state=42, subsample=0.8))])

In [11]:
y_pred_lgbm = clf_lgbm.predict(X_test)

print('LightGBM Classifier Accuracy:', accuracy_score(y_test, y_pred_lgbm))
print('\nClassification Report:\n', classification_report(y_test, y_pred_lgbm))
print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_pred_lgbm))

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


LightGBM Classifier Accuracy: 0.6711086703783979

Classification Report:
               precision    recall  f1-score   support

           1       0.02      0.46      0.03       166
           2       0.97      0.67      0.79    149447
           3       0.33      0.70      0.45     13959
           4       0.15      0.67      0.24      7597

    accuracy                           0.67    171169
   macro avg       0.37      0.63      0.38    171169
weighted avg       0.88      0.67      0.74    171169


Confusion Matrix:
 [[   76    40    23    27]
 [ 3605 99873 18585 27384]
 [  359  1546  9831  2223]
 [  144  1302  1058  5093]]
